In [1]:
#Importamos la librerias necesarias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



###Buscamos crear un clasificador, que dado una muestra de aire nos diga de que Ciudad viene la muestra

In [2]:
#Importamos la base de datos en el archivo csv que vamos a utilizar (De la ciudad 1)
from google.colab import files
files.upload()
df1 = pd.read_csv("PRSA_Data_Aotizhongxin_20130301-20170228.csv")

#df.fillna(0)
#Donde hay nan ponemos un 0.0
df1=df1.replace(np.nan, 0.0)
#print(df1)

Saving PRSA_Data_Aotizhongxin_20130301-20170228.csv to PRSA_Data_Aotizhongxin_20130301-20170228.csv


In [3]:
print(df1)

          No  year  month  day  hour  ...  DEWP  RAIN   wd  WSPM       station
0          1  2013      3    1     0  ... -18.8   0.0  NNW   4.4  Aotizhongxin
1          2  2013      3    1     1  ... -18.2   0.0    N   4.7  Aotizhongxin
2          3  2013      3    1     2  ... -18.2   0.0  NNW   5.6  Aotizhongxin
3          4  2013      3    1     3  ... -19.4   0.0   NW   3.1  Aotizhongxin
4          5  2013      3    1     4  ... -19.5   0.0    N   2.0  Aotizhongxin
...      ...   ...    ...  ...   ...  ...   ...   ...  ...   ...           ...
35059  35060  2017      2   28    19  ... -16.2   0.0   NW   2.4  Aotizhongxin
35060  35061  2017      2   28    20  ... -15.1   0.0  WNW   0.9  Aotizhongxin
35061  35062  2017      2   28    21  ... -13.3   0.0   NW   1.1  Aotizhongxin
35062  35063  2017      2   28    22  ... -12.9   0.0  NNW   1.2  Aotizhongxin
35063  35064  2017      2   28    23  ... -15.9   0.0  NNE   1.3  Aotizhongxin

[35064 rows x 18 columns]


In [4]:
#Importamos la base de datos en el archivo csv que vamos a utilizar (De la ciudad 2)

files.upload()
df2 = pd.read_csv("PRSA_Data_Changping_20130301-20170228.csv")

#df.fillna(0)
#Donde hay nan ponemos un 0.0
df2=df2.replace(np.nan, 0.0)
#print(df2)

Saving PRSA_Data_Changping_20130301-20170228.csv to PRSA_Data_Changping_20130301-20170228.csv


In [5]:
print(df2)

          No  year  month  day  hour  ...  DEWP  RAIN   wd  WSPM    station
0          1  2013      3    1     0  ... -19.7   0.0    E   0.5  Changping
1          2  2013      3    1     1  ... -19.0   0.0  ENE   0.7  Changping
2          3  2013      3    1     2  ... -19.9   0.0  ENE   0.2  Changping
3          4  2013      3    1     3  ... -19.1   0.0  NNE   1.0  Changping
4          5  2013      3    1     4  ... -19.4   0.0    N   2.1  Changping
...      ...   ...    ...  ...   ...  ...   ...   ...  ...   ...        ...
35059  35060  2017      2   28    19  ... -13.3   0.0  NNE   1.3  Changping
35060  35061  2017      2   28    20  ... -14.0   0.0    N   2.1  Changping
35061  35062  2017      2   28    21  ... -13.0   0.0    N   1.5  Changping
35062  35063  2017      2   28    22  ... -12.6   0.0   NW   1.4  Changping
35063  35064  2017      2   28    23  ... -12.2   0.0    N   1.9  Changping

[35064 rows x 18 columns]


In [6]:
frames = [df1, df2]

###Juntamos ambos dataFrames para tener las dos ciudades en uno
dataFrameFinal = pd.concat(frames)






In [7]:
print(dataFrameFinal)
###Mezclamos de manera aleatoria las filas
dataFrameFinal.sample(frac=1)

del dataFrameFinal["wd"]
del dataFrameFinal["No"]
for col in dataFrameFinal.columns:
    print(col)

          No  year  month  day  hour  ...  DEWP  RAIN   wd  WSPM       station
0          1  2013      3    1     0  ... -18.8   0.0  NNW   4.4  Aotizhongxin
1          2  2013      3    1     1  ... -18.2   0.0    N   4.7  Aotizhongxin
2          3  2013      3    1     2  ... -18.2   0.0  NNW   5.6  Aotizhongxin
3          4  2013      3    1     3  ... -19.4   0.0   NW   3.1  Aotizhongxin
4          5  2013      3    1     4  ... -19.5   0.0    N   2.0  Aotizhongxin
...      ...   ...    ...  ...   ...  ...   ...   ...  ...   ...           ...
35059  35060  2017      2   28    19  ... -13.3   0.0  NNE   1.3     Changping
35060  35061  2017      2   28    20  ... -14.0   0.0    N   2.1     Changping
35061  35062  2017      2   28    21  ... -13.0   0.0    N   1.5     Changping
35062  35063  2017      2   28    22  ... -12.6   0.0   NW   1.4     Changping
35063  35064  2017      2   28    23  ... -12.2   0.0    N   1.9     Changping

[70128 rows x 18 columns]
year
month
day
hour
PM2.5

In [8]:
###Manejaremos a la ciudad Aotizhongxin como 0.0 y a Changping como 1.0

dataFrameFinal1 = dataFrameFinal.replace("Aotizhongxin", 0.0)
data = dataFrameFinal1.replace("Changping", 1.0)
data = data.sample(frac = 1)


In [9]:
print(data)

       year  month  day  hour  PM2.5  ...    PRES  DEWP  RAIN  WSPM  station
27435  2016      4   17     3   11.0  ...  1007.2  -9.0   0.0   3.7      0.0
11484  2014      6   22    12   41.0  ...   999.4  16.0   0.0   1.7      1.0
14957  2014     11   14     5   41.0  ...  1017.0  -4.6   0.0   0.7      0.0
10199  2014      4   29    23    0.0  ...  1005.2   9.2   0.0   2.4      0.0
7295   2013     12   29    23   94.0  ...  1011.5 -17.1   0.0   2.9      0.0
...     ...    ...  ...   ...    ...  ...     ...   ...   ...   ...      ...
1336   2013      4   25    16   10.0  ...  1001.8 -10.8   0.0   3.2      1.0
29468  2016      7   10    20   74.0  ...   996.7  19.2   0.0   2.3      0.0
27446  2016      4   17    14    7.0  ...  1001.2  -7.7   0.0   5.3      1.0
29927  2016      7   29    23  102.0  ...  1002.8  26.1   0.0   2.0      0.0
13645  2014      9   20    13  100.0  ...   999.7  14.8   0.0   1.8      1.0

[70128 rows x 16 columns]


In [10]:
###Asignamos los features a la variable X (todos menos las ciudades o no)
X= data.iloc[:,0:-1]
print("X es lo siguiente")
print(X)
###Asignamos las ciudades a la variable Y
print("Y es lo siguinte")
Y= data.iloc[:,-1]
print(Y)


X es lo siguiente
       year  month  day  hour  PM2.5  ...  TEMP    PRES  DEWP  RAIN  WSPM
27435  2016      4   17     3   11.0  ...  13.6  1007.2  -9.0   0.0   3.7
11484  2014      6   22    12   41.0  ...  26.7   999.4  16.0   0.0   1.7
14957  2014     11   14     5   41.0  ...  -1.6  1017.0  -4.6   0.0   0.7
10199  2014      4   29    23    0.0  ...  19.9  1005.2   9.2   0.0   2.4
7295   2013     12   29    23   94.0  ...   3.2  1011.5 -17.1   0.0   2.9
...     ...    ...  ...   ...    ...  ...   ...     ...   ...   ...   ...
1336   2013      4   25    16   10.0  ...  22.3  1001.8 -10.8   0.0   3.2
29468  2016      7   10    20   74.0  ...  33.5   996.7  19.2   0.0   2.3
27446  2016      4   17    14    7.0  ...  18.7  1001.2  -7.7   0.0   5.3
29927  2016      7   29    23  102.0  ...  29.7  1002.8  26.1   0.0   2.0
13645  2014      9   20    13  100.0  ...  26.7   999.7  14.8   0.0   1.8

[70128 rows x 15 columns]
Y es lo siguinte
27435    0.0
11484    1.0
14957    0.0
10199    0.

In [11]:
###Dividimos nuestros X y Y en 

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33,random_state=69)
print(y_train)

22312    1.0
28895    0.0
10010    1.0
14748    0.0
22955    1.0
        ... 
22279    1.0
27747    1.0
17184    1.0
13994    1.0
19060    0.0
Name: station, Length: 46985, dtype: float64


In [12]:
###Definimos el standardScaler para estandarizar los features para train y test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
###Definimos epocas, bathsize y learning rate:
###Las epocas nos indican cuantas veces se ha completado el algoritmo de entrenamiento
EPOCHS = 10
###De cuantos en cuantos datos iremos tomando de nuestra data set
BATCH_SIZE = 70
###El leraning rate nos dice que tan rapido se va a adaptar nuestro modelo al problema
LEARNING_RATE = 0.001

In [14]:
##Train data
##Clase train data
class trainData(Dataset):
    
    ###Al contructor e pasamos la X_data y y_data
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    ###Para sacar un item de la data como una tupla
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    ###Longitud de la x data
    def __len__ (self):
        return len(self.X_data)


#Inicializamos el objeto train_data, le pasamos el X_train y y_train como objetos tipo tensor, esto para
#poder trbajar con la redes nuronales usando pytorch
###AQui es donde a veces se da el error
train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data
###Lo mismo que la train data pero cuando lo construyamos la vamos a pasar la X_test data
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

###Inicializamos el testData y le pasamos la X_test data como tensor
test_data = testData(torch.FloatTensor(X_test))

In [15]:
###Creamos los objetos train_loader y test_loader con los objetos Dataloader que trae pytorch, en el train data 
#le pasamos el batch size previamente defindido y le decimos que mezcle los objetos
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [16]:

###Usamos binary clasification por que el problema solo tiene dos soluciones
#Viene de la ciudad 0, o no
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()       
         # Como el numero de features es 15 y queremos 100 neuronas en la primera capa ponemos 15,100
        self.layer_1 = nn.Linear(15, 100) 
        ###A la segunda capa entran 100 y salen 100
        self.layer_2 = nn.Linear(100, 100)
        ###En la capa final solo necesitamos una salida (si es la ciudad1  o no), por eso el 1
        self.layer_out = nn.Linear(100, 1) 
        
        ###Que la funcion de activacion sea la funcion relu
        self.relu = nn.ReLU()

        ###Dropout para que durante el entrenamiento, este pone a cero aleatoriamente a algunos de los elementos del 
        ###tensor de entrada con probabilidad p usando muestras de una distribución de Bernoulli.
        self.dropout = nn.Dropout(p=0.1)
        ### BatchNorm1d aplica la normalización por lotes sobre una entrada 2D o 3D
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.batchnorm2 = nn.BatchNorm1d(100)
        

    ###Aplicamos la funcion de activacion relu y batchnorm a la primera y segunda capa, luego dropout y final,
    ###esta funcion se ve asi por la cantidad de capas que tiene nuestra red
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [17]:
###Verificamos que todo lo este mandando a la gpu para que el proceso sea mas rapido
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [18]:
###Construimos el modelo
model = binaryClassification()
###Buscamos mandar los datos a la gpu para que las multiplicaciones y sumas se realizen mas rapido
model.to(device)
print(model)
###De criterio vamos a usar una BCEWithLogitsLoss, esta pérdida combina una capa sigmoidea y BCELoss en una sola clase.
criterion = nn.BCEWithLogitsLoss()
###Usamos el optimzador Adam
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

binaryClassification(
  (layer_1): Linear(in_features=15, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=100, bias=True)
  (layer_out): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [19]:

###Funcion binary_acc para ver como va la accuracy de nuestro modelo en cada epoca
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [20]:
###Le decimos al modelo que se va a poner en modoentrenamiento
model.train()
###Hacemos un for la cantidad de epocas especificadas
###Las variables epoch_loss e epoch_acc se iran actualizando en cada iteracion, lo que buscamos es que
#la variable de perdida baje en cada iteracion y la de accuracy suba en cada iteracion
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)# Se envian los datos a entrenar a la GPU que proporciona Colab de google
        optimizer.zero_grad()#Cada vez que una variable realiza el backpropagation, el gradiente se acumulará en lugar de ser reemplazado.
        #Dado que la función backward () acumula gradientes y no deseamos mezclar gradientes entre minibatches, 
        #debe ponerse a cero al comienzo de un nuevo minibatch. 
        #Es una mejor práctica el usar la función zero_grad() en el optimizador 
        
        y_pred = model(X_batch)#se hace una prueba(test) con un nuevo minibatch 
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))#se obtiene la funcion de perdida en funcion de los resultados previos 
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))#se obtiene la precision de este minibatch 
        
        loss.backward()#Se realiza el algoritmo de gradiente descendiente para encontrar los parametros o pesos optimos
        optimizer.step()#Se actualizan dichos pesos en el modelo
        #Se acumulan tanto la funcion de perdida como la precisión de que tiene el modelo para verificar el desempeño del entrenamiento
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.52998 | Acc: 73.039
Epoch 002: | Loss: 0.45139 | Acc: 78.482
Epoch 003: | Loss: 0.42774 | Acc: 79.658
Epoch 004: | Loss: 0.40748 | Acc: 80.938
Epoch 005: | Loss: 0.39435 | Acc: 81.731
Epoch 006: | Loss: 0.37623 | Acc: 82.826
Epoch 007: | Loss: 0.36525 | Acc: 83.415
Epoch 008: | Loss: 0.35600 | Acc: 83.844
Epoch 009: | Loss: 0.34537 | Acc: 84.451
Epoch 010: | Loss: 0.34041 | Acc: 84.966
Epoch 011: | Loss: 0.33137 | Acc: 85.244
Epoch 012: | Loss: 0.32886 | Acc: 85.493
Epoch 013: | Loss: 0.31983 | Acc: 85.987
Epoch 014: | Loss: 0.31365 | Acc: 86.247
Epoch 015: | Loss: 0.31076 | Acc: 86.448
Epoch 016: | Loss: 0.30607 | Acc: 86.527
Epoch 017: | Loss: 0.30004 | Acc: 86.912
Epoch 018: | Loss: 0.29635 | Acc: 87.189
Epoch 019: | Loss: 0.29685 | Acc: 87.177
Epoch 020: | Loss: 0.29401 | Acc: 87.277
Epoch 021: | Loss: 0.28643 | Acc: 87.589
Epoch 022: | Loss: 0.28541 | Acc: 87.653
Epoch 023: | Loss: 0.28171 | Acc: 88.144
Epoch 024: | Loss: 0.27740 | Acc: 88.253
Epoch 025: | Los